In [1]:
!pip install numpy
!pip install pyomo 

import numpy as np
import pandas as pd
from pyomo.environ import *
import time  # Importing the time module to track runtime


np.random.seed(30)

# Start the timer to track runtime
start_time = time.time()

# Sets
I_df = pd.read_excel("TrialDataNCDOT.xlsx")  # Charging station locations
J_df = pd.read_excel("TrialDataNCDOT.xlsx")   # Demand locations
K_df = pd.read_excel("power_substation.xlsx")        # Power substations

# Ensure columns are consistently named
I_df.columns = ["Latitude", "Longitude"]
J_df.columns = ["Latitude", "Longitude"]
K_df.columns = ["Latitude", "Longitude"]

I_df["Latitude"] = pd.to_numeric(I_df["Latitude"], errors="coerce")
I_df["Longitude"] = pd.to_numeric(I_df["Longitude"], errors="coerce")

J_df["Latitude"] = pd.to_numeric(J_df["Latitude"], errors="coerce")
J_df["Longitude"] = pd.to_numeric(J_df["Longitude"], errors="coerce")

K_df["Latitude"] = pd.to_numeric(K_df["Latitude"], errors="coerce")
K_df["Longitude"] = pd.to_numeric(K_df["Longitude"], errors="coerce")

# Add unique identifiers for rows
I_df['ID'] = ['station' + str(i + 1) for i in range(len(I_df))]
J_df['ID'] = ['demand' + str(j + 1) for j in range(len(J_df))]
K_df['ID'] = ['substation' + str(k + 1) for k in range(len(K_df))]

# Create sets
I = I_df['ID'].tolist()  # Charging station IDs
J = J_df['ID'].tolist()  # Demand location IDs
K = K_df['ID'].tolist()  # Substation IDs

# Parameters
d_j = {row['ID']: np.random.randint(20, 29) for _, row in J_df.iterrows()}  # Random demands for testing
#d_j = {row['ID']: 1000 for _, row in J_df.iterrows()}  # Random demands for testing
C_i = {row['ID']: 100 for _, row in I_df.iterrows()}  # Example capacities
P_k = {row['ID']: 150 for _, row in K_df.iterrows()}  # Substation capacities
S_k = {row['ID']: 50 for _, row in K_df.iterrows()}  # Existing substation demand

# Euclidean distance function
def euclidean(lat1, lon1, lat2, lon2):
    return np.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)

# Compute distance-based binary parameters a_ij and b_ik
threshold_ij = 100  # Coverage threshold
threshold_ik = 100  # Substation feasibility threshold

a_ij = {}
for _, row_i in I_df.iterrows():
    for _, row_j in J_df.iterrows():
        dist = euclidean(row_i['Latitude'], row_i['Longitude'], row_j['Latitude'], row_j['Longitude'])
        a_ij[(row_i['ID'], row_j['ID'])] = 1 if dist <= threshold_ij else 0

b_ik = {}
for _, row_i in I_df.iterrows():
    for _, row_k in K_df.iterrows():
        dist = euclidean(row_i['Latitude'], row_i['Longitude'], row_k['Latitude'], row_k['Longitude'])
        b_ik[(row_i['ID'], row_k['ID'])] = 1 if dist <= threshold_ik else 0

# Create the Pyomo model
model = ConcreteModel()

# Decision Variables
model.x = Var(I, domain=Binary)  # Charging station decision
model.y = Var(J, domain=Binary)  # Demand coverage decision
model.z = Var([(i, k) for i in I for k in K], domain=Binary)  # Substation-station connection decision

# Objective Function: Maximize demand coverage
model.obj = Objective(expr=sum(d_j[j] * model.y[j] for j in J), sense=maximize)

# Constraints

# 1. Budget constraint
Q = 80000 # Total budget
F = 2000   # Fixed cost
model.budget = Constraint(expr=sum(model.x[i] for i in I) <= Q / F)

# 2. Coverage constraint
def coverage_rule(model, j):
    return model.y[j] <= sum(a_ij[i, j] * model.x[i] for i in I)
model.coverage = Constraint(J, rule=coverage_rule)

# 3. Charging station capacity constraint
def station_capacity_rule(model, i):
    return sum(a_ij[i, j] * d_j[j] * model.y[j] for j in J) <= C_i[i] * model.x[i]
model.station_capacity = Constraint(I, rule=station_capacity_rule)

# 4. Substation capacity constraint
def substation_capacity_rule(model, k):
    return sum(model.z[i, k] * C_i[i] for i in I) + S_k[k] <= P_k[k]
model.substation_capacity = Constraint(K, rule=substation_capacity_rule)

# 5. Substation feasibility constraint
def feasibility_rule(model, i, k):
    return model.z[i, k] <= b_ik[i, k]
model.feasibility = Constraint([(i, k) for i in I for k in K], rule=feasibility_rule)



# Solve the model
solver = SolverFactory('glpk')  # Use an open-source solver like CBC
solver.solve(model)

# End the timer to track runtime
end_time = time.time()
runtime = end_time - start_time

# Count the number of open stations
num_open_stations = sum(1 for i in I if model.x[i]() > 0.5)

# Output results
print("Objective value (Total demand covered):", model.obj())

print("\nCharging station placement:")
for i in I:
    print(f"  Station {i}: {'Open' if model.x[i]() > 0.5 else 'Closed'}")

print("\nDemand coverage:")
for j in J:
    print(f"  Demand {j}: {'Covered' if model.y[j]() > 0 else 'Not Covered'}")

print("\nSubstation connections:")
for i in I:
    for k in K:
        if model.z[i, k]() > 0:
            print(f"  Station {i} is connected to Substation {k}")

# Print the number of open stations and runtime
print(f"\nNumber of open stations: {num_open_stations}")
print(f"Model runtime: {runtime:.2f} seconds")


Objective value (Total demand covered): 22.0

Charging station placement:
  Station station1: Open
  Station station2: Open
  Station station3: Open
  Station station4: Open
  Station station5: Open
  Station station6: Open
  Station station7: Open
  Station station8: Open
  Station station9: Open
  Station station10: Open
  Station station11: Open
  Station station12: Open
  Station station13: Open
  Station station14: Open
  Station station15: Open
  Station station16: Open
  Station station17: Open
  Station station18: Open
  Station station19: Open
  Station station20: Open
  Station station21: Open
  Station station22: Open
  Station station23: Open
  Station station24: Open
  Station station25: Open
  Station station26: Open
  Station station27: Open
  Station station28: Open
  Station station29: Open
  Station station30: Open
  Station station31: Open
  Station station32: Open
  Station station33: Closed
  Station station34: Closed
  Station station35: Closed
  Station station36